## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [3]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [4]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,-0.119973,-0.119229,0.002337,0.069874
1,-0.147773,0.084399,0.060399,-0.029892
2,-0.174988,-0.004611,0.009333,-0.050979
3,-0.148295,0.092466,0.036414,-0.038096
4,-0.169614,0.080814,0.043074,-0.031368


In [5]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00299657', u'USC00292810', u'US1NMSM0005', u'US1NMBR0053', u'US1NMSM0007']


,0,1,2,3
station,,,,
USC00299657,-0.119973,-0.119229,0.002337,0.069874
USC00292810,-0.147773,0.084399,0.060399,-0.029892
US1NMSM0005,-0.174988,-0.004611,0.009333,-0.050979
US1NMBR0053,-0.148295,0.092466,0.036414,-0.038096
US1NMSM0007,-0.169614,0.080814,0.043074,-0.031368


In [6]:
table=Eig.join(Stations,how='left')

In [7]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00299657,WESTBROOK RCH,35.9333,-108.2000,2012.9,-0.119973,-0.119229,0.002337,0.069874
USC00292810,EL PUEBLO PROJECT,35.2833,-105.2667,1708.1,-0.147773,0.084399,0.060399,-0.029892
US1NMSM0005,LAS VEGAS 8.4 NW,35.6680,-105.3423,2292.1,-0.174988,-0.004611,0.009333,-0.050979
US1NMBR0053,ALBUQUERQUE 3.1 ENE,35.1377,-106.5767,1610.9,-0.148295,0.092466,0.036414,-0.038096


In [8]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Additional Analysis

In [30]:
table

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00299657,WESTBROOK RCH,35.9333,-108.2000,2012.9,-0.119973,-0.119229,0.002337,0.069874
USC00292810,EL PUEBLO PROJECT,35.2833,-105.2667,1708.1,-0.147773,0.084399,0.060399,-0.029892
US1NMSM0005,LAS VEGAS 8.4 NW,35.6680,-105.3423,2292.1,-0.174988,-0.004611,0.009333,-0.050979
US1NMBR0053,ALBUQUERQUE 3.1 ENE,35.1377,-106.5767,1610.9,-0.148295,0.092466,0.036414,-0.038096
US1NMSM0007,LAS VEGAS 2.6 SE,35.5704,-105.1889,1976.3,-0.169614,0.080814,0.043074,-0.031368
US1NMSM0006,LAS VEGAS 1.6 NNW,35.6198,-105.2342,2003.5,-0.133813,0.028969,0.069933,0.027002
US1NMBR0056,TIJERAS 3.7 N,35.1427,-106.3695,2118.4,-0.174506,-0.035345,-0.007789,-0.060141
US1NMSM0002,LAS VEGAS 13 NNE,35.8000,-105.3270,2239.1,-0.138324,-0.045217,-0.087919,-0.063803
USS0006P10S,SENORITA DIVIDE #2,36.0000,-106.8333,2621.3,-0.142341,0.032600,0.113816,0.041788


In [40]:
len(Res_eig['stations'])

186

## Map

In [ ]:
min_lat,max_lat,min_long,max_long = box = (42.1103, 42.6167, -72.6, -70.8)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    print 1
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [ ]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');